## Regression Trading Strategy in Python

The principle of regression trading strategy is as follows. Let's say you have a combination of stocks in your portfolio that have some underlying economic link. An assumption to be made here is that they are always linked to each other in the same degree. If there is a deviation in this relationship, we trade them such that we assume that they will come back.

In [1]:
import numpy as np
import pandas as pd
import statsmodels
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint, adfuller
import warnings
import plotly.express as px
import yfinance as yf
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import plotly.graph_objects as go
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style="whitegrid")

c:\Users\OuiWeinJien\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## Helper Function

### Stationary test , Extracting data

In [2]:
def stationarity_test(X, cutoff=0.15):
    # H_0 in adfuller is unit root exists (non-stationary)
    # We must observe significant p-value to convince ourselves that the series is stationary
    pvalue = adfuller(X)[1]
    if pvalue < cutoff:
        print('p-value = ' + str(pvalue) + ' The series is likely stationary.')
    else:
        print('p-value = ' + str(pvalue) + ' The series is likely non-stationary.')

In [3]:
def get_multiple_stock_data(tickers, start_date, end_date, interval='1d'):
    """
    Fetch historical 'Close' stock data for multiple tickers from Yahoo Finance.

    Parameters:
    tickers (list of str): List of stock ticker symbols (e.g., ['AAPL', 'MSFT']).
    start_date (str): Start date in the format 'YYYY-MM-DD'.
    end_date (str): End date in the format 'YYYY-MM-DD'.
    interval (str): Data interval ('1d', '1wk', '1mo').

    Returns:
    pd.DataFrame: DataFrame containing the 'Close' prices for each ticker.
    """
    data = yf.download(tickers, start=start_date, end=end_date, interval=interval)['Close']
    return data

In [4]:
# Regression on three stock prices (ADSK and EA on TXN) using OLS
def regression_on_three_stocks(df):
    X = df.iloc[:,:-1]
    Y = df.iloc[:,-1]
    X = sm.add_constant(X)
    model = sm.OLS(Y, X).fit()
    spread = Y - (model.params[1] * df.iloc[:,0] + model.params[2] * df.iloc[:,1] + model.params[0])
    return model.rsquared, spread, model.params

In [5]:
## Helper function:
# Compute the spread and z-score
def compute_spread_and_zscore(df, params):
    spread = df.iloc[:,2] - (params[1] * df.iloc[:,0] + params[2] * df.iloc[:,1] + params[0])
    zscore = (spread - np.mean(spread)) / np.std(spread)
    return spread, zscore

In [6]:
# Step 3: Function to calculate the longest streak of True values
def longest_streak(boolean_array):
    max_streak = 0
    current_streak = 0
    for value in boolean_array[0]:
        if value:
            current_streak += 1
            max_streak = max(max_streak, current_streak)
        else:
            current_streak = 0
    return max_streak

## Calculate number of consetsutive days the z-score is above or below the threshold
def max_drawback(data, weightage_list, zscore):
    
    daily_capital = weightage_list[0]* np.mean(data.iloc[:,0]) + weightage_list[1] * np.mean(data.iloc[:,1]) + weightage_list[2] * np.mean(data.iloc[:,2])
    time_series = np.array([zscore])

    # Step 2: Identify if each day is above or below the mean
    above_mean = time_series > 0
    below_mean = time_series < 0

    # Calculate the longest streak of days above the mean
    longest_above_streak = longest_streak(above_mean)

    # Calculate the longest streak of days below the mean
    longest_below_streak = longest_streak(below_mean)
    return max(longest_above_streak, longest_below_streak)* abs(daily_capital)

In [7]:
def trade_spread(zscore, data, weightage, initial_capital, initial_position, alpha, clear_position_end = False):
    ## Initial capital
    ## Params: weightage, data (stocks)
    capital = initial_capital
    capital_progress = []
    count_stock_1 = initial_position[0]
    count_stock_2 = initial_position[1]
    count_stock_3 = initial_position[2]
    
    # alpha = capital/max_drawback(data,weightage,zscore) #(enlargement factor)
    print('alpha',alpha)
    action_threshold = 1
    clear_position_threshold = 0.1
    stop_loss_threshold = 2

    #Check
    count1 = []
    count2 = []
    count3 = []
    action_description = []
    current_zscore = []
    value_list = []
    action = 0
    EOD_position=0 ### add by Shihan 
    return_daily=[] ### add by Shihan 



    
    # Simulate trading
    for i in range(len(zscore)):
        ### add by Shihan
        
        Open_position=count_stock_1 * data.iloc[i,0] + count_stock_2 * data.iloc[i,1] + count_stock_3 * data.iloc[i,2] 
        return_in_cash=Open_position-EOD_position 
        if EOD_position==0: 
            return_in_percentage=0 
        else:
            return_in_percentage=return_in_cash/abs(EOD_position) 
        return_daily.append(return_in_percentage) 
        
        ### add by Shihan
        
        # If previously bought and zscore < 0, clear the portfolio; if previously sold and zscore > 0, clear the portfolio
        if (action > 0 and zscore[i] < clear_position_threshold) or (action < 0 and zscore[i] > -clear_position_threshold):
            capital += count_stock_1 * data.iloc[i,0] + count_stock_2 * data.iloc[i,1] + count_stock_3 * data.iloc[i,2]
            count_stock_1 = 0
            count_stock_2 = 0
            count_stock_3 = 0
        # If zscore > 1, sell the portfolio
        if (zscore[i] > action_threshold) and (np.abs(zscore[i]) < stop_loss_threshold):
            capital += weightage[0]*alpha * data.iloc[i,0] + weightage[1] *alpha* data.iloc[i,1] + weightage[2] *alpha* data.iloc[i,2]
            count_stock_1 -= weightage[0]*alpha
            count_stock_2 -= weightage[1]*alpha
            count_stock_3 -= weightage[2]*alpha
            capital_progress.append(capital)
            count1.append(count_stock_1)
            count2.append(count_stock_2)
            count3.append(count_stock_3)
            current_zscore.append(zscore[i])
            action_description.append('Sell')
            action += 1
        # If zscore < -1, buy the portfolio
        elif (zscore[i] < -action_threshold) and (np.abs(zscore[i]) < stop_loss_threshold):
            capital -= weightage[0] *alpha* data.iloc[i,0] + weightage[1] *alpha* data.iloc[i,1] + weightage[2] *alpha* data.iloc[i,2]
            count_stock_1 += weightage[0]*alpha
            count_stock_2 += weightage[1]*alpha
            count_stock_3 += weightage[2]*alpha
            capital_progress.append(capital)
            count1.append(count_stock_1)
            count2.append(count_stock_2)
            count3.append(count_stock_3)
            current_zscore.append(zscore[i])
            action_description.append('Buy')
            action += -1
        # If trade volume is 0 AND z-score between -.5 and .5, clear the position
        elif (np.abs(zscore[i]) < clear_position_threshold):
            capital += count_stock_1 * data.iloc[i,0] + count_stock_2 * data.iloc[i,1] + count_stock_3 * data.iloc[i,2]
            count_stock_1 = 0
            count_stock_2 = 0
            count_stock_3 = 0
            capital_progress.append(capital)
            count1.append(count_stock_1)
            count2.append(count_stock_2)
            count3.append(count_stock_3)
            action_description.append('Clear Position')
            action = 0
        elif (np.abs(zscore[i]) >= stop_loss_threshold):
            capital += count_stock_1 * data.iloc[i,0] + count_stock_2 * data.iloc[i,1] + count_stock_3 * data.iloc[i,2]
            count_stock_1 = 0
            count_stock_2 = 0
            count_stock_3 = 0
            capital_progress.append(capital)
            count1.append(count_stock_1)
            count2.append(count_stock_2)
            count3.append(count_stock_3)
            action_description.append('Stop Loss')
            action = 0
        else:
            capital_progress.append(capital)
            count1.append(count_stock_1)
            count2.append(count_stock_2)
            count3.append(count_stock_3)
            action_description.append('Hold')
            action += 0
        value = capital + count_stock_1 * data.iloc[i,0] + count_stock_2 * data.iloc[i,1] + count_stock_3 * data.iloc[i,2]
        value_list.append(value)

        EOD_position=count_stock_1 * data.iloc[i,0] + count_stock_2 * data.iloc[i,1] + count_stock_3 * data.iloc[i,2] ### add by Shihan

    
    if clear_position_end == True:
        # Clear all positions at the end
        capital += count_stock_1 * data.iloc[-1,0] + count_stock_2 * data.iloc[-1,1] + count_stock_3 * data.iloc[-1,2]
        count_stock_1 = 0
        count_stock_2 = 0
        count_stock_3 = 0 
        capital_progress.append(capital)
        count1.append(count_stock_1)
        count2.append(count_stock_2)
        count3.append(count_stock_3)
        action_description.append('Hold')
        capital_progress = pd.Series(capital_progress)
    return capital_progress, count1, count2, count3, zscore, action_description, value_list, return_daily

In [8]:
def trade_train_zscore(train_data, initial_capital, initial_position, alpha = True, clear_position_end = False):
    # Regression on three stock prices (ADSK and EA on TXN) using OLS
    r, spread, param = regression_on_three_stocks(train_data)
    
    # Compute the spread and z-score
    zscore_train = (spread - np.mean(spread)) / np.std(spread)
    
    ## Convert coefficient to weightage
    weightage = [-param[1], -param[2], 1]
    
    if alpha == True:
        alpha = initial_capital/max_drawback(train_data,weightage,zscore_train)
    else: 
        alpha = alpha
    
    # Trade the spread
    capital_progress, count1, count2, count3, zscore, action_description, value_list, return_daily = trade_spread(zscore_train, train_data, weightage, initial_capital, initial_position, alpha, clear_position_end)
   
    return capital_progress, count1, count2, count3, zscore, action_description, value_list, return_daily, r

In [9]:
def trade_test_zscore(train_data, test_data, initial_capital, initial_position, alpha= True):
    ## Obtain weightage from train_date
    r, spread, param = regression_on_three_stocks(train_data)
    
    z_score_train = (spread - np.mean(spread)) / np.std(spread)

    print("R-squared: ",r)
    ## Compute spread and zscore for test data using param from train data
    spread_test, zscore_test = compute_spread_and_zscore(test_data, param)

    ## Convert coefficient to weightage
    weightage = [-param[1], -param[2], 1]
    
    if alpha == True:
        alpha = initial_capital/max_drawback(train_data,weightage,z_score_train)
    else:
        alpha = alpha
    ## Trade spread
    capital_progress, count1, count2, count3, _, action, value_list, return_daily = trade_spread(zscore_test, test_data, weightage, initial_capital, initial_position,alpha, False)
    return capital_progress, count1, count2, count3, value_list, zscore_test, weightage, action, return_daily, r



In [10]:
def compute_volatility(data, weights):
    # Ensure the weights and data are pandas DataFrames
    if not isinstance(data, pd.DataFrame) or not isinstance(weights, pd.DataFrame):
        raise ValueError("Both data and weights must be pandas DataFrames.")
    
    # Compute the covariance matrix of asset returns
    cov_matrix = np.cov(data, rowvar=False)
    
    assert cov_matrix.shape[0] == 9, "Please select 9 stocks"
    
    # Convert DataFrames to numpy arrays
    weights_matrix = weights.values  # Shape: (t, a)
    
    assert weights_matrix.shape[1] == 9, "Please select 9 stocks"
    assert weights_matrix.shape[0] == data.shape[0], "The number of rows in the weights matrix must match the number of rows in the data DataFrame."
    
    # Initialize an array to hold volatilities for each time point
    num_time_points = weights_matrix.shape[0]
    volatilities = np.zeros(num_time_points)
    
    # Compute portfolio volatility for each time point
    for t in range(num_time_points):
        weight_vector = weights_matrix[t, :]  # Shape: (a,)
        # Portfolio variance = weight_vector^T * cov_matrix * weight_vector
        portfolio_variance = np.dot(weight_vector.T, np.dot(cov_matrix, weight_vector))
        volatilities[t] = np.sqrt(portfolio_variance)    
    return volatilities


In [11]:
import statistics
def sharpe_ratio(return_daily):
    mean_value = statistics.mean(return_daily) ##add by SH
    std_dev = statistics.stdev(return_daily) ##add by SH
    if std_dev == 0:
        return 0
    else:
        return mean_value/std_dev ##add by SH
    

## Choose the best stock to trade

### 1. To Look at Train Sharpe Ratio & R-squared

In [13]:
#### Train add by Shihan
import ast
start = datetime(2023, 1, 1)
end = datetime(2024, 7, 30)
train_date_end = datetime(2024,1,1)
capital = 1000000
position = (0,0,0)
Return_Matrix_train = pd.DataFrame()

Sharpe_ratio_train = {}
R_squared_train = {}

Triplet_Selections = pd.read_csv('triplets_after_coefficient_tests_revised.csv')
# Triplet_Selections = pd.read_excel('triplets_after_coefficient_tests_revised.csv', sheet_name='Sheet1')
tickers_list_size=len(Triplet_Selections['tickers'])

for i in range (1, tickers_list_size):
    combinations = ast.literal_eval(Triplet_Selections['tickers'][i])
    #print(combinations)
    
    df = get_multiple_stock_data(combinations, start, end)
    df_train = df.loc[:train_date_end - relativedelta(days=1)]

    capital_progress_train_1, count1, count2, count3, zcore_train_1, action, value_list_train_1, return_daily_train_1, r1_train = trade_train_zscore(df_train, capital, position)
    #print("Return 1", np.mean(return_daily_train_1))
    #print("Sharpe Ratio 1", np.mean(return_daily_train_1)/np.std(return_daily_train_1))
    
    column_name = f'"{combinations}"'
    Return_Matrix_train[column_name] = return_daily_train_1
    Sharpe_ratio_train[column_name] = sharpe_ratio(return_daily_train_1)
    R_squared_train[column_name] = r1_train

Return_Matrix_train

[*********************100%%**********************]  3 of 3 completed
[**********************67%%******                ]  2 of 3 completed

alpha 121.1828073613308


[*********************100%%**********************]  3 of 3 completed
[*********************100%%**********************]  3 of 3 completed


alpha 674.5745608628893
alpha 83.9443591647751


[*********************100%%**********************]  3 of 3 completed
[*********************100%%**********************]  3 of 3 completed


...

[*********************100%%**********************]  3 of 3 completed
[*********************100%%**********************]  3 of 3 completed


alpha 8182.4930227554405
alpha 26732.37612439277


[*********************100%%**********************]  3 of 3 completed
[*********************100%%**********************]  3 of 3 completed


alpha 3335.004476170143
alpha 4287.9787132490865


,"""['AAON', 'BYU', 'LIN']""","""['AAON', 'CSWI', 'STRL']""","""['AAON', 'AGRI', 'RETO']""","""['AAON', 'BNGO', 'OMIC']""","""['AAON', 'AKYA', 'OMIC']""","""['AAON', 'AKYA', 'MNTS']""","""['AAOI', 'AAPL', 'MANH']""","""['AAOI', 'ALKT', 'VRNS']""","""['AAOI', 'ALKT', 'NTNX']""","""['AAOI', 'AAPL', 'TTWO']""",...,"""['AADI', 'ACET', 'NUWE']""","""['AADI', 'ABCL', 'BACK']""","""['AADI', 'ABVC', 'PYPD']""","""['AADI', 'ACET', 'KZR']""","""['AADI', 'ACB', 'KZR']""","""['AADI', 'ACHL', 'KZR']""","""['AADI', 'ABOS', 'INGN']""","""['AADI', 'ABEO', 'INGN']""","""['AADI', 'ACHL', 'INGN']""","""['AADI', 'ACET', 'CARA']"""
0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
1,0.000000,0.059938,0.0,0.000000,0.000000,-0.232705,0.000000,0.000000,0.000000,-0.197114,...,0.000000,0.000000,0.000000,-0.148983,0.030167,0.036528,0.157312,0.0,0.0,0.000000
2,0.000000,0.012821,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,-0.091401,0.219758,0.016263,0.065896,0.117065,0.0,0.0,0.000000
3,0.034074,0.053096,0.0,0.000000,0.000000,0.000000,0.000000,0.092094,0.000000,0.000000,...,0.000000,0.000000,0.264709,0.107506,-0.000598,0.065763,0.058935,0.0,0.0,0.000000
4,0.009072,0.008986,0.0,0.000000,0.000000,0.000000,0.000000,0.013863,0.000000,0.021552,...,0.000000,0.000000,0.247571,0.143461,0.116244,0.012881,0.147242,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,0.000000,-0.017376,0.0,-0.010683,0.000942,0.000000,0.026456,0.086212,0.087650,0.000000,...,0.030740,0.000000,0.082196,0.054159,0.000000,0.000000,0.000000,0.0,0.0,-0.030340
246,0.000000,-0.002644,0.0,0.051081,0.028113,0.000000,-0.060842,0.051587,0.101592,0.000000,...,0.035178,-0.530460,0.466640,0.028461,0.000000,0.000000,0.000000,0.0,0.0,0.013507
247,0.000000,0.016317,0.0,-0.019149,-0.016944,0.000000,-0.044435,-0.032021,0.000000,0.000000,...,0.049576,0.000000,-0.242854,-0.111406,-0.051772,0.000000,0.000000,0.0,0.0,0.074911
248,0.000000,0.028875,0.0,-0.037236,-0.034225,0.000000,0.010023,-0.046642,0.000000,0.000000,...,0.002202,0.364767,0.311416,-0.015370,0.000000,0.000000,0.000000,0.0,0.0,-0.053635


### 2. To Look at Test R-squared

In [14]:
#### Test add by Shihan 
import ast
capital = 1000000
position = (0,0,0)
date_july = datetime(2024,7,1)
start = datetime(2023, 1, 1)
end = datetime(2024, 7, 30)


Triplet_Selections = pd.read_csv('triplets_after_coefficient_tests_revised.csv')
#Triplet_Selections = pd.read_excel('triplets_after_coefficient_tests_revised.xlsx', sheet_name='Sheet1')
tickers_list_size=len(Triplet_Selections['tickers'])

R_squared_test = {}
sector = {}

for i in range (1, tickers_list_size):
    combinations = ast.literal_eval(Triplet_Selections['tickers'][i])
    #print(combinations)
    
    df = get_multiple_stock_data(combinations, start, end)

    train_data = df.loc[date_july - relativedelta(months = 12):date_july - relativedelta(minutes=1)]
    test_data = df.loc[date_july:]
    capital_progress_test_1, count1, count2, count3, value_list_test_1, zscore_test_1, w_1,action,return_daily_test_1,r1_test = trade_test_zscore(train_data, test_data, capital, position)
    

    column_name = f'"{combinations}"'
    
    R_squared_test[column_name] = r1_test
    sector[column_name] = Triplet_Selections['sector'][i]

[*********************100%%**********************]  3 of 3 completed
[*********************100%%**********************]  3 of 3 completed
[*********************100%%**********************]  3 of 3 completed

R-squared:  0.8473321450070156
alpha 84.27249694324323
R-squared:  0.8357463754661251
alpha 809.2576911835251



[*********************100%%**********************]  3 of 3 completed
[*********************100%%**********************]  3 of 3 completed
[                       0%%                      ]

R-squared:  0.36639800657786425
alpha 2545.06799854972
R-squared:  0.7462207143331734
alpha 4387.005981276496
R-squared:  0.7156297572267286
alpha 12973.399775501059


[*********************100%%**********************]  3 of 3 completed
[*********************100%%**********************]  3 of 3 completed
[**********************67%%******                ]  2 of 3 completed

...

R-squared:  0.03410362937784983
alpha 7535.912620901736
R-squared:  0.6486828437875234
alpha 6359.707557933053
R-squared:  0.7869475071313792
alpha 44794.30067498663


[*********************100%%**********************]  3 of 3 completed
[*********************100%%**********************]  3 of 3 completed
[*********************100%%**********************]  3 of 3 completed
[*********************100%%**********************]  3 of 3 completed


R-squared:  0.48962910567630313
alpha 27452.37321373299
R-squared:  0.5616440088592869
alpha 29516.606963653056
R-squared:  0.16888657037078336
alpha 2290.970785879452
R-squared:  0.041765231592647
alpha 1352.0489748434222


[*********************100%%**********************]  3 of 3 completed
[*********************100%%**********************]  3 of 3 completed

R-squared:  0.15006661650339648
alpha 3416.245761866351
R-squared:  0.834421279781618
alpha 21711.678425290324


### 3. Look at overall performance

In [15]:
overall = pd.DataFrame([Sharpe_ratio_train, R_squared_train, R_squared_test,sector]).transpose()
overall.columns = ['Sharpe Ratio Train', 'R Squared Train', 'R Squared Test','sector']
overall.sort_values(by='Sharpe Ratio Train', ascending=False).head(20)

,Sharpe Ratio Train,R Squared Train,R Squared Test,sector
"""['AACI', 'ABCB', 'ORRF']""",0.27127,0.932771,0.940806,Finance
"""['AACI', 'ABCB', 'THFF']""",0.254669,0.949822,0.879611,Finance
"""['AACI', 'ABCB', 'WAFD']""",0.248253,0.936979,0.754073,Finance
"""['AADI', 'ABEO', 'SCNI']""",0.24356,0.904547,0.770809,Health-Care
"""['AADI', 'ABUS', 'SCNI']""",0.238398,0.915288,0.781747,Health-Care
"""['AAON', 'BYU', 'LIN']""",0.233415,0.906579,0.847332,Industrials
"""['AACI', 'ABCB', 'BWFG']""",0.231822,0.950647,0.8354,Finance
"""['AACI', 'ABCB', 'LKFN']""",0.230362,0.906252,0.861965,Finance
"""['AACI', 'ABCB', 'ATLO']""",0.230325,0.933859,0.725017,Finance
"""['AACI', 'ABCB', 'ESSA']""",0.226887,0.926881,0.899703,Finance


### 4. Apply conditions to choose stocks

In [17]:
condition1 = overall['R Squared Train'] > 0.9
condition2 = overall['R Squared Test'] > 0.8
condition3 = overall['Sharpe Ratio Train'] > 0.1

chosen = overall[condition1 & condition2  & condition3]
chosen

,Sharpe Ratio Train,R Squared Train,R Squared Test,sector
"""['AAON', 'BYU', 'LIN']""",0.233415,0.906579,0.847332,Industrials
"""['AAOI', 'ALKT', 'VRNS']""",0.12389,0.907314,0.871865,Technology
"""['AAL', 'ADSE', 'MNRO']""",0.126397,0.908631,0.8718,Consumer-Discretionary
"""['AAL', 'ADSE', 'ULCC']""",0.108865,0.922391,0.810463,Consumer-Discretionary
"""['AACI', 'ABCB', 'MCBS']""",0.10521,0.921624,0.925172,Finance
"""['AACI', 'ACGLN', 'ACGLO']""",0.157725,0.902047,0.930397,Finance
"""['AACI', 'ABCB', 'ORRF']""",0.27127,0.932771,0.940806,Finance
"""['AACI', 'ABCB', 'SSBK']""",0.205262,0.922001,0.859661,Finance
"""['AACI', 'ABCB', 'THFF']""",0.254669,0.949822,0.879611,Finance
"""['AACI', 'ABCB', 'ESSA']""",0.226887,0.926881,0.899703,Finance


## Action to trade

When spread is more than 0.5 sigma (std), we make an trade action. This is due to the fact that there is a consideration on transaction cost in reality. Otherwise, Ideally the trading strategy would be trade whenever there is a spread.

In this case, let's suppose we choose the combinations of ***META, AVGO, LRCX***.

In [18]:
start = datetime(2023, 1, 1)
end = datetime(2024, 7, 30)

train_date_end = datetime(2024,1,1)
valid_date_end = datetime(2024,7,1)

all_tickers = ['AAOI', 'ALKT', 'VRNS','AAL', 'ADSE', 'ULCC','AACI', 'ABCB', 'ORRF']

tickers = ['AAON', 'BYU', 'LIN']
sector = ['Consumer', 'Technology', 'Healthcare']

#['AADI', 'ABUS', 'SILK'] - Healthcare
#['AAOI', 'AAPL', 'LRCX'] - Technology
#['AAL', 'AMZN', 'WINA'] - Consumer

df = get_multiple_stock_data(all_tickers, start, end)
df = df[all_tickers]
df_train = df.loc[:train_date_end - relativedelta(days=1)]
df_valid = df.loc[train_date_end:valid_date_end - relativedelta(days=1)]
df_test = df.loc[valid_date_end:]
df_valid_test = df.loc[train_date_end:]

[*********************100%%**********************]  9 of 9 completed


## Determine weight for each triplet in portfolio


In [19]:
print('Assuming intial capital for each sector is 1000000')
r1 = trade_train_zscore(df_train.iloc[:,0:3], 1000000, (0,0,0))
print(f"Train profit for sector {sector[0]}: {r1[-3][-1]}")

r2 = trade_train_zscore(df_train.iloc[:,3:6], 1000000, (0,0,0))
print(f"Train profit for sector {sector[1]}: {r2[-3][-1]}")

r3 = trade_train_zscore(df_train.iloc[:,6:9], 1000000, (0,0,0))
print(f"Train profit for sector {sector[2]}: {r3[-3][-1]}")

w1 = r1[-3][-1]
w2 = r2[-3][-1]
w3 = r3[-3][-1]
print(w1,w2,w3)
w1p = w1/(w1+w2+w3)
w2p = w2/(w1+w2+w3)
w3p = w3/(w1+w2+w3)
print(w1p, w2p, w3p)

Assuming intial capital for each sector is 1000000
alpha 2745.009493347819
Train profit for sector Consumer: 1222947.2234215224
alpha 13323.851842897408
Train profit for sector Technology: 1502614.3662225239
alpha 2576.4022448301603
Train profit for sector Healthcare: 1117332.9798858433
1222947.2234215224 1502614.3662225239 1117332.9798858433
0.3182359550319717 0.3910110826710351 0.29075296229699304


## Train set result

In [20]:
capital = 1000000
position = (0,0,0)
capital_progress_train_1, count1, count2, count3, zcore_train_1, action, value_list_train_1, return_daily_train_1, r1_train = trade_train_zscore(df_train.iloc[:,0:3], capital*w1p, position)
capital_progress_train_2, count4, count5, count6, zcore_train_2, action, value_list_train_2, return_daily_train_2, r2_train = trade_train_zscore(df_train.iloc[:,3:6], capital*w2p, position)
capital_progress_train_3, count7, count8, count9, zcore_train_3, action, value_list_train_3, return_daily_train_3, r3_train = trade_train_zscore(df_train.iloc[:,6:9], capital*w3p, position)

print("Initial capital", capital*w1p + capital*w2p + capital*w3p)
capital_progress_train = np.sum([capital_progress_train_1, capital_progress_train_2, capital_progress_train_3], axis=0)
assert len(capital_progress_train) == len(df_train)
value_list_train = np.sum([value_list_train_1, value_list_train_2, value_list_train_3], axis=0)
return_daily_train = np.sum([return_daily_train_1, return_daily_train_2, return_daily_train_3], axis=0)
r_train = [r1_train, r2_train, r3_train]
count = pd.DataFrame(np.array([count1, count2, count3, count4, count5, count6, count7, count8, count9]).T)
volatility_train = compute_volatility(df_train, count)
print("R squared", r_train)
print("final Capital", value_list_train[-1])
print("Sharpe Ratio 1", np.mean(return_daily_train_1)/np.std(return_daily_train_1))
print("Sharpe Ratio 2", np.mean(return_daily_train_2)/np.std(return_daily_train_2))
print("Sharpe Ratio 3", np.mean(return_daily_train_3)/np.std(return_daily_train_3))

#add by Shihan 
print("Volatility",np.std(return_daily_train))

print("Daily Return",np.mean(return_daily_train))
print("Annualized Return", np.mean(return_daily_train)*252) #simple return
      
print("Daily Sharpe Ratio", np.mean(return_daily_train)/np.std(return_daily_train))
print("Annualized Sharpe Ratio", np.mean(return_daily_train*252)/(np.std(return_daily_train)*np.sqrt(252))) ## daily return is independent, so portfolio var is 252*daily var     


#print("Volatility", np.mean(volatility_train))

alpha 873.5607176873721
alpha 5209.773734439782
alpha 749.0965847529919
Initial capital 1000000.0
R squared [0.9073135444224628, 0.9223908264957374, 0.9327714451541478]
final Capital 1301592.5215469028
Sharpe Ratio 1 0.12413869487990269
Sharpe Ratio 2 0.10908361316516546
Sharpe Ratio 3 0.27181448849535705
Volatility 0.4549824663791915
Daily Return 0.06210602717845451
Annualized Return 15.650718848970536
Daily Sharpe Ratio 0.13650202319377755
Annualized Sharpe Ratio 2.1669024409674367


## Validation set result (with varying coefficients)

In [21]:
# Validation set
# Loop through 12 months since 2024 with month start date
date_range = pd.date_range(start='2023-01-01', end='2024-07-01', freq='MS')

capital = 1000000
capital1 = capital*w1p
capital2 = capital*w2p
capital3 = capital*w3p

position1 = (0,0,0)
position2 = (0,0,0)
position3 = (0,0,0)
total_capital_progress_valid = []
value_list_valid = []
zscore_list_valid = []
weightage_list = []
date_index = []
return_daily_valid = []
r_valid_list = []
volatility_valid_list = []
for i in range(len(date_range)-1):
    if date_range[i] >= train_date_end:
        train_data = df.loc[date_range[i-12]:date_range[i] - relativedelta(minutes=1)]
        test_data = df.loc[date_range[i]:date_range[i+1]- relativedelta(minutes=1)]
        capital_progress_valid_1, count1, count2, count3, value_list_1, zscore_valid_1, w_1,action,return_daily_1,r1_valid = trade_test_zscore(train_data.iloc[:,0:3], test_data.iloc[:,0:3], capital1, position1, alpha = 873)
        capital_progress_valid_2, count4, count5, count6, value_list_2, zscore_valid_2, w_2,action,return_daily_2,r2_valid = trade_test_zscore(train_data.iloc[:,3:6], test_data.iloc[:,3:6], capital2, position2, alpha = 5209)
        capital_progress_valid_3, count7, count8, count9, value_list_3, zscore_valid_3, w_3,action,return_daily_3,r3_valid = trade_test_zscore(train_data.iloc[:,6:9], test_data.iloc[:,6:9], capital3, position3, alpha = 750)
        #print(len(capital_progress_valid_1), len(capital_progress_valid_2), len(capital_progress_valid_3))
        position1 = (count1[-1], count2[-1], count3[-1])
        position2 = (count4[-1], count5[-1], count6[-1])
        position3 = (count7[-1], count8[-1], count9[-1])
        capital1 = capital_progress_valid_1[-1]
        capital2 = capital_progress_valid_2[-1]
        capital3 = capital_progress_valid_3[-1]    
        
        
        total_capital_progress_valid += list(np.sum([capital_progress_valid_1, capital_progress_valid_2, capital_progress_valid_3], axis=0))
        value_list_valid += list(np.sum([value_list_1, value_list_2, value_list_3], axis=0))
        zscore_list_valid += list(np.sum([zscore_valid_1, zscore_valid_2, zscore_valid_3], axis=0))
        weightage_list.append([w_1,w_2,w_3])
        date_index += test_data.index.tolist()
        return_daily_valid += list(np.sum([return_daily_1, return_daily_2, return_daily_3], axis=0))
        r_valid_list.append([r1_valid, r2_valid, r3_valid])
        count = pd.DataFrame(np.array([count1, count2, count3, count4, count5, count6, count7, count8, count9]).T)
        volatility_valid = compute_volatility(test_data, count)
        volatility_valid_list += list(volatility_valid)
        #print('Adjusted weightage!')
        print('Profit as of month', date_range[i], value_list_valid[-1])
r_valid_list = np.array(r_valid_list)
r_valid_list = np.mean(r_valid_list, axis=0)
print("Total profit", value_list_valid[-1])
#print("Sharpe Ratio", np.mean(return_daily_valid)/np.std(return_daily_valid))
#print("Volatility valid", np.mean(volatility_valid_list))

###add by Shihan 
print("Volatility",np.std(return_daily_valid))

print("Daily Return",np.mean(return_daily_valid))
print("Annualized Return", np.mean(return_daily_valid)*252) #simple return
      
print("Daily Sharpe Ratio", np.mean(return_daily_valid)/np.std(return_daily_valid))
print("Annualized Sharpe Ratio", np.mean(return_daily_valid*252)/(np.std(return_daily_valid)*np.sqrt(252))) ## daily return is independent, so portfolio var is 252*daily var     



R-squared:  0.9073135444224628
alpha 873
R-squared:  0.9223908264957374
alpha 5209
R-squared:  0.9327714451541478
alpha 750
Profit as of month 2024-01-01 00:00:00 1031031.6664642154
R-squared:  0.9304056384755947
alpha 873
R-squared:  0.9169243174154185
alpha 5209
R-squared:  0.9468654279102753
alpha 750
Profit as of month 2024-02-01 00:00:00 1063110.3211359144
R-squared:  0.9375737592063652
alpha 873
R-squared:  0.83995912992975
alpha 5209
R-squared:  0.9605724584011917
alpha 750
Profit as of month 2024-03-01 00:00:00 1081037.8386368216
R-squared:  0.9280665242491644
alpha 873
R-squared:  0.7051989493840165
alpha 5209
R-squared:  0.970512973217948
alpha 750
Profit as of month 2024-04-01 00:00:00 1107034.187119947
R-squared:  0.9368735123971516
alpha 873
R-squared:  0.7444579018797086
alpha 5209
R-squared:  0.9693113605664718
alpha 750
Profit as of month 2024-05-01 00:00:00 1128565.5496209543
R-squared:  0.9031692534836773
alpha 873
R-squared:  0.8045076684490372
alpha 5209
R-squared: 

## Test set result

In [22]:
capital = 1000000
position = (0,0,0)

date_july = datetime(2024,7,1)
train_data = df.loc[date_july - relativedelta(months = 12):date_july - relativedelta(minutes=1)]
test_data = df.loc[date_july:]
capital_progress_test_1, count1, count2, count3, value_list_test_1, zscore_test_1, w_1,action,return_daily_test_1,r1_test = trade_test_zscore(train_data.iloc[:,0:3], test_data.iloc[:,0:3], capital*w1p, position, alpha = 873)
capital_progress_test_2, count4, count5, count6, value_list_test_2, zscore_test_2, w_2,action,return_daily_test_2,r2_test = trade_test_zscore(train_data.iloc[:,3:6], test_data.iloc[:,3:6], capital*w2p, position, alpha = 5209)
capital_progress_test_3, count7, count8, count9, value_list_test_3, zscore_test_3, w_3,action,return_daily_test_3,r3_test = trade_test_zscore(train_data.iloc[:,6:9], test_data.iloc[:,6:9], capital*w3p, position, alpha = 750)

print("Initial capital", capital*w1p + capital*w2p + capital*w3p)
capital_progress_test = np.sum([capital_progress_test_1, capital_progress_test_2, capital_progress_test_3], axis=0)
assert len(capital_progress_test) == len(df_test)
value_list_test = np.sum([value_list_test_1, value_list_test_2, value_list_test_3], axis=0)
return_daily_test = np.sum([return_daily_test_1, return_daily_test_2, return_daily_test_3], axis=0)
r_test = [r1_test, r2_test, r3_test]

count = pd.DataFrame(np.array([count1, count2, count3, count4, count5, count6, count7, count8, count9]).T)
volatility_test = compute_volatility(test_data, count)
print("R squared", r_test)
print("final Capital", value_list_test[-1])
print("Sharpe Ratio 1", np.mean(return_daily_test_1)/np.std(return_daily_test_1))
print("Sharpe Ratio 2", np.mean(return_daily_test_2)/np.std(return_daily_test_2))
print("Sharpe Ratio 3", np.mean(return_daily_test_3)/np.std(return_daily_test_3))
print("Sharpe Ratio", np.mean(return_daily_test)/np.std(return_daily_test))
#print("Volatility test", np.mean(volatility_test))


###add by Shihan 
print("Volatility",np.std(return_daily_test))

print("Daily Return",np.mean(return_daily_test))
print("Annualized Return", np.mean(return_daily_test)*252) #simple return
      
print("Daily Sharpe Ratio", np.mean(return_daily_test)/np.std(return_daily_test))
print("Annualized Sharpe Ratio", np.mean(return_daily_test*252)/(np.std(return_daily_test)*np.sqrt(252))) ## daily return is independent, so portfolio var is 252*daily var     


R-squared:  0.8718649766189422
alpha 873
R-squared:  0.8104627500389892
alpha 5209
R-squared:  0.9408059358266601
alpha 750
Initial capital 1000000.0
R squared [0.8718649766189422, 0.8104627500389892, 0.9408059358266601]
final Capital 1040144.9217056148
Sharpe Ratio 1 0.3584587203763022
Sharpe Ratio 2 0.2622223709402193
Sharpe Ratio 3 0.3911364099453592
Sharpe Ratio 0.5106915570713647
Volatility 0.3615553465874794
Daily Return 0.18464326291623678
Annualized Return 46.530102254891666
Daily Sharpe Ratio 0.5106915570713647
Annualized Sharpe Ratio 8.106977140027082


## End Summary of Result (Train，Validation, Test)



In [23]:
from IPython.display import Markdown

print_md = lambda x: display(Markdown(x))

print_md(f"""
# Final Result

### Stocks selected: **{all_tickers}**

### Training set

Profit Margin: **{((value_list_train[-1]/(capital*w1p + capital*w2p + capital*w3p)) - 1)*100:.2f}%**

r-squared: **{r_train}**


Daily Return:**{np.mean(return_daily_train)*100:.2f} %**

Annual Return: **{np.mean(return_daily_train)*252*100:.2f} %**


Volatility: **{ np.std(return_daily_train)*100:.2f} %**

      
Daily Sharpe Ratio: **{ np.mean(return_daily_train)/np.std(return_daily_train)*100:.2f} %**

Annualized Sharpe Ratio: **{ np.mean(return_daily_train)*252/(np.std(return_daily_train)*np.sqrt(252))*100:.2f} %**


### Validation set

Profit Margin: **{((value_list_valid[-1]/(capital*w1p + capital*w2p + capital*w3p))-1)*100:.2f}%**

r-squared: **{r_valid_list}**


Daily Return:**{np.mean(return_daily_valid)*100:.2f} %**

Annual Return: **{np.mean(return_daily_valid)*252*100:.2f} %**


Volatility: **{ np.std(return_daily_valid)*100:.2f} %**

      
Daily Sharpe Ratio: **{ np.mean(return_daily_valid)/np.std(return_daily_valid)*100:.2f} %**

Annualized Sharpe Ratio: **{ 100*np.mean(return_daily_valid)*252/(np.std(return_daily_valid)*np.sqrt(252)):.2f} %**




### Test set

Profit Margin: **{(value_list_test[-1]/(capital*w1p + capital*w2p + capital*w3p)-1)*100:.2f}%**

r-squared: **{r_test}**


Daily Return:**{np.mean(return_daily_test)*100:.2f} %**

Annual Return: **{np.mean(return_daily_test)*252*100:.2f} %**


Volatility: **{ np.std(return_daily_test)*100:.2f} %**

      
Daily Sharpe Ratio: **{ np.mean(return_daily_test)/np.std(return_daily_test)*100:.2f} %**

Annualized Sharpe Ratio: **{ np.mean(return_daily_test)*252/(np.std(return_daily_test)*np.sqrt(252))*100:.2f} %**


""")


# Final Result

### Stocks selected: **['AAOI', 'ALKT', 'VRNS', 'AAL', 'ADSE', 'ULCC', 'AACI', 'ABCB', 'ORRF']**

### Training set

Total Profit: **1,301,593**

Profit Margin: **30.16%**

r-squared: **[0.9073135444224628, 0.9223908264957374, 0.9327714451541478]**


Daily Return:**6.21 %**

Annual Return: **1565.07 %**


Volatility: **45.50 %**

      
Daily Sharpe Ratio: **13.65 %**

Annualized Sharpe Ratio: **216.69 %**


### Validation set

Total Profit: **1,148,697**

Profit Margin: **14.87%**

r-squared: **[0.92390037 0.8222398  0.95598835]**


Daily Return:**13.91 %**

Annual Return: **3505.59 %**


Volatility: **50.43 %**

      
Daily Sharpe Ratio: **27.59 %**

Annualized Sharpe Ratio: **437.94 %**




### Test set

Total Profit: **1,040,145**

Profit Margin: **4.01%**

r-squared: **[0.8718649766189422, 0.8104627500389892, 0.9408059358266601]**


Daily Return:**18.46 %**

Annual Return: **4653.01 %**


Volatility: **36.16 %**

      
Daily Sharpe Ratio: **51.07 %**

Annualized Sharpe Ratio: **810.70 %**




## Appendix (Plotting)

# Train plot

In [345]:
px.line(x = df_train.index, y = value_list_train, title = 'Capital Progress')

In [346]:
px.line(x = df_train.index, y = volatility_train, title = 'daily_return')

## Validation Plot

In [347]:
px.line(x = df_valid.index, y = total_capital_progress_valid, title = 'Capital Progress')

In [348]:
px.line(x = date_index,y = volatility_valid_list, title = 'Total Value (Cash + current stock value)').show()

## Test Plot

In [349]:
px.line(x = test_data.index, y = capital_progress_test, title = 'Capital TEST').show()

In [350]:
px.line(x = test_data.index,y = volatility_test, title = 'Volatility').show()

## To-do:

1. Try different sector (Jiajun)
2. Explain background, sector of stocks (Adrian, Shi Han)
3. Try train test (Wayne)
4. Explain why we choose 3 stocks from 3 sectors (Shi Han)
5. Discuss how number of stocks effect profit & risk - Why we fix three stocks?
6. Risk management 
    - Add a checking that there is no significant portion invested on single stock? (Mingrui)
    - Filter, Add a layer of checking standard deviation of coefficient (weightage) (Mingrui)
    - T-Copula (low priority, Jiajun/Wayne/Mingrui)
    - TimeSeries of Sharpe ratio of portfolio(ideally > 2) (Shi Han)
7. Add a enlargement factor - consider the case where you have a lot of capital (Wayne)
8. Discuss brainstorm initial capital, rate of return (Group)
9. Compute profit for top 5 R-squared - act as 2nd layer when choosing stock. (wayne)
10. Discuss sell (short) how it operates in real world market. (Shi Han)
11. Discuss adjusting your weightage based on rolling window. (Adrian)
12. Implement stop-loss order (Wayne)

Pair trading with extended version:
1.	Story: Market needs, Literature (Adrian)
2.	Implementation (Shi han)
3.	Result interpretation (Ming rui)
4.	Risk management (Wayne)
5.	Assumption, Area of improvement, Limitation (Jiajun)
    - When to execute
    - Short sell
    - Transaction cost
6.  Executive summary (Adrian)

### Handling Risks of pair trading
1. Reassess the Pair Trade
Review the Fundamentals: Check if there are any fundamental changes in the assets that might have caused the weakening correlation.
Market Conditions: Consider if current market conditions or external factors are temporarily affecting the pair relationship.
2. Adjust Your Positions
Reduce Exposure: Gradually reduce your positions to limit potential losses. You can sell part of your holdings to decrease your risk exposure.
Close Positions: If the R-squared value falls below a critical threshold that you have set for your strategy, it might be wise to close the positions entirely.
3. Monitor and Adjust Strategy
Set Alerts: Implement alerts to notify you when the R-squared value falls below certain levels, so you can react promptly.
Rolling Recalibration: Continuously monitor the rolling R-squared and spread to make informed decisions on adjusting your strategy.
4. Implement Risk Management Techniques
Stop-Loss Orders: Use stop-loss orders to automatically exit positions if the spread widens beyond a certain point.
Hedging: Consider hedging your positions with other financial instruments to mitigate potential losses.

To